# Zielsetzung

- Emojis in Tweets spielen wichtige Rolle: spart Zeichen
- User nutzen z. B. Flaggen-Emojis, um das umzusetzen: man muss nicht "Deutschland" ausschreiben
- Deswegen: Emojis tragen eine Botschaft, genauso wie auch normaler Text
- Studien befassten sich z. B. auch NUR mit der Emoji-Nutzung auf Twitter, deshalb auch notwendig für hiesige Arbeit

## Packages

In [1]:
import pandas as pd
import numpy as np
import demoji
import emoji
import string
from functools import reduce
import re
import pprint

## Datensatz

In [2]:
politikR_updated = pd.read_csv(filepath_or_buffer="../../methods/representative-speech/regionalisierung/rest/politikR_updated.csv")

In [3]:
politikR_updated.content[995]

'Wer mit Geflüchteten aus der 🇺🇦 spricht, weiß, unter welchen extremen Belastungen sie stehen. Sie verdienen unsere volle Solidarität und Hilfe. Wer von #Sozialtourismus spricht, möge nach Butscha, Irpin, Isjum schauen. Schlimmer Populismus, der auch hier zu Hass und Hetze führt.'

In [4]:
demoji.findall(politikR_updated.content[995])

{'🇺🇦': 'flag: Ukraine'}

## Funktionen

In [5]:
regex = r"-(?!\w)|(?<!\w)-"
regex2 = r"–(?!\w)|(?<!\w)–"
regex2 = r"–(?!\w)|(?<!\w)–"

text = "ich teste es – das 9-euro-ticket."

print(re.sub(regex, "", text))
print(re.sub(regex2, "", text))

ich teste es – das 9-euro-ticket.
ich teste es  das 9-euro-ticket.


In [6]:
## aus einer liste einige zeichen entfernen
def remChar(lst, char):
    return reduce(lambda x, y: x + [re.sub(char, '', y)], lst, [])

string.punctuation

zeichen = remChar(string.punctuation, '@')
zeichen = remChar(zeichen, '-')

print("Volles Set aller Satzzeichen ---> " + string.punctuation + "\n")
print("Reduziertes Set, Bindestriche und @ behalten: ")
print(zeichen)
print(type(zeichen))

Volles Set aller Satzzeichen ---> !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

Reduziertes Set, Bindestriche und @ behalten: 
['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '', '.', '/', ':', ';', '<', '=', '>', '?', '', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
<class 'list'>


In [7]:
nan = ''

for i in zeichen:
    result = str(i or nan)
type(zeichen)
print(len(zeichen))

while('' in zeichen):
    zeichen.remove('')
    
zeichen.append("„")
zeichen.append("“")

print(zeichen)
print(len(zeichen))


32
['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '.', '/', ':', ';', '<', '=', '>', '?', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '„', '“']
32


In [8]:
# aus einem objekt (dataframe) innerhalb von i zeichen entfernen, die nicht teil eines frozen sets sind
def remPunct(s):
    s = ''.join([i for i in s if i not in frozenset(zeichen)])
    return s

## Datenbearbeitung

### Funktion für einen Aufwasch

In [9]:
def preprocessing(dataframe, i, savename):
    print("Ausgangs-String: \n" + dataframe.content[i] + "\n")
    
    ## operationen
    dataframe['content'] = dataframe['content'].apply(lambda x: emoji.demojize(x, delimiters=(' ',' ')))
    dataframe['text'] = dataframe['content'].apply(remPunct)
    dataframe['text'] = dataframe['content'].apply(lambda x: x.lower())
    dataframe['text'] = dataframe['content'].apply(lambda x: re.sub(regex, "", x))
    dataframe['text'] = dataframe['content'].apply(lambda x: re.sub(regex2, "", x))
    
    dataframe.to_csv(f"../../methods/representative-speech/regionalisierung/demojized/{savename}.csv")
    return print("Endprodukt preprocessing: \n" + dataframe.text[i])

In [10]:
preprocessing(politikR_updated, 8725, "politikR_demojized")

Ausgangs-String: 
Endlich🎆 wieder Politik in der Hauptstadt!
Morgen:
Drei(!) Veranstaltungen beim #TagderoffenenTür, ab Montag dann #Sitzungswoche - mit #AfD-pur!💪
#FürdieBürger✌️
#Brandnerunterwegs
#AfD
#wirhabendasDirektmandat👍
#Deutschlandabernormal🇩🇪
#WK194 #Berlin #Bundestag #Brandner https://t.co/OOpoIu7i3f

Endprodukt preprocessing: 
Endlich fireworks  wieder Politik in der Hauptstadt!
Morgen:
Drei(!) Veranstaltungen beim #TagderoffenenTür, ab Montag dann #Sitzungswoche - mit #AfD-pur! flexed_biceps 
#FürdieBürger victory_hand 
#Brandnerunterwegs
#AfD
#wirhabendasDirektmandat thumbs_up 
#Deutschlandabernormal Germany 
#WK194 #Berlin #Bundestag #Brandner https://t.co/OOpoIu7i3f


In [9]:
print("Bsp. 1: " + politikR_updated.content[1234] + "\n")

# demojize
politikR_updated['content'] = politikR_updated['content'].apply(lambda x: emoji.demojize(x, delimiters=('', '')))

# interpunktion entfernen
politikR_updated['text'] = politikR_updated['content'].apply(remPunct)

# kleinbuchstaben
politikR_updated['text'] = politikR_updated['text'].apply(lambda x: x.lower())

# problematik bindestriche: nur extra-word, nicht intra-word --- manche nutzen "-" zwischen wörtern, manche "–"
politikR_updated['text'] = politikR_updated['text'].apply(lambda x: re.sub(regex, "", x))
politikR_updated['text'] = politikR_updated['text'].apply(lambda x: re.sub(regex2, "", x))

print("Bsp. 1: " + politikR_updated.text[1234] + "\n")

Bsp. 1: Darf man als Bundestagsabgeordneter über Ostern Urlaub machen? 
Ich frage nur für einen Freund. #AnneSpiegel

Bsp. 1: darf man als bundestagsabgeordneter über ostern urlaub machen 
ich frage nur für einen freund annespiegel



In [10]:
print("1: " + politikR_updated.text[1] + "\n")

print("2: " + politikR_updated.text[42]+ "\n")

print("3: " + politikR_updated.text[995]+ "\n")

print("4: " + politikR_updated.text[100947]+ "\n")

1: was @janboehm und das @zdfmagazin da gemacht haben ist einfach so unfassbar krass es legt den finger in eine wunde die auch nach 120 jahren nicht verheilt wäre 

danke dafür

2: @nadjasthamer @bmzbund lt3

3: wer mit geflüchteten aus der ukraine spricht weiß unter welchen extremen belastungen sie stehen sie verdienen unsere volle solidarität und hilfe wer von sozialtourismus spricht möge nach butscha irpin isjum schauen schlimmer populismus der auch hier zu hass und hetze führt

4: bisher tragen mieterinnen die co²-preise alleine die fdp will dass es so bleibt spdgrüne wollen kosten aufteilen als linke sagen wir wenn die co²-preise nicht abgeschafft werden dann dürfen sie wenigstens nicht den mieterinnen aufgebürdet werden httpstcoqqtzatbrsl httpstco9dkzwwhy1l



In [11]:
politikR_updated.to_csv("../../methods/representative-speech/regionalisierung/demojized/politikR_demoji.csv")